<a href="https://colab.research.google.com/github/gigikenneth/self-supervised/blob/main/ssl1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define your self-supervised learning model architecture
class SelfSupervisedModel(nn.Module):
    def __init__(self, num_classes):
        super(SelfSupervisedModel, self).__init__()
        # Define your model layers here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(64 * 100 * 100, num_classes)

    def forward(self, x):
        # Define the forward pass of your model here
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# Set random seed for reproducibility
torch.manual_seed(42)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
num_epochs = 10
batch_size = 32
learning_rate = 0.001
gamma = 0.1  # Learning rate decay factor
step_size = 5  # Number of epochs after which to decay learning rate

# Define your data transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image tensors
])

# Set the path to your unlabelled images directory
unlabelled_images_dir = '/content/drive/MyDrive/images'

# Define the custom dataset class for unlabelled images
class UnlabelledDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image

# Create the unlabelled dataset
unlabelled_dataset = UnlabelledDataset(unlabelled_images_dir, transform=transform)

# Create a data loader for your unlabelled dataset
unlabelled_dataloader = DataLoader(unlabelled_dataset, batch_size=batch_size, shuffle=True)

# Define your self-supervised learning model
model = SelfSupervisedModel(num_classes=10).to(device)

# Define your loss function (e.g., contrastive loss or other self-supervised learning loss)
criterion = nn.CrossEntropyLoss()

# Get the trainable parameters of the model
parameters = filter(lambda p: p.requires_grad, model.parameters())
parameters = list(parameters)  # Convert to a list

# Check if there are trainable parameters
if len(parameters) == 0:
    raise ValueError("No trainable parameters found in the model.")

# Define your optimizer
optimizer = optim.Adam(parameters, lr=learning_rate)

# Define your learning rate scheduler
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_idx, images in enumerate(unlabelled_dataloader):
        images = images.to(device)

        # Forward pass
        outputs = model(images)

        # Generate placeholder targets (since the images are unlabelled)
        targets = torch.zeros(images.size(0), dtype=torch.long).to(device)

        # Compute the self-supervised learning loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss after every few iterations
        if (batch_idx+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(unlabelled_dataloader)}], Loss: {loss.item():.4f}')

    # Update the learning rate scheduler
    scheduler.step()

    # Display epoch information
    print(f"End of Epoch {epoch+1}. Learning Rate: {scheduler.get_last_lr()[0]}")

# End of training
print("Training finished.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch [1/10], Step [10/124], Loss: 0.0000
Epoch [1/10], Step [20/124], Loss: 0.0000
Epoch [1/10], Step [30/124], Loss: 0.0000
Epoch [1/10], Step [40/124], Loss: 0.0000
Epoch [1/10], Step [50/124], Loss: 0.0000
Epoch [1/10], Step [60/124], Loss: 0.0000
Epoch [1/10], Step [70/124], Loss: 0.0000
Epoch [1/10], Step [80/124], Loss: 0.0000
Epoch [1/10], Step [90/124], Loss: 0.0000
Epoch [1/10], Step [100/124], Loss: 0.0000
Epoch [1/10], Step [110/124], Loss: 0.0000
Epoch [1/10], Step [120/124], Loss: 0.0000
End of Epoch 1. Learning Rate: 0.001
Epoch [2/10], Step [10/124], Loss: 0.0000
Epoch [2/10], Step [20/124], Loss: 0.0000
Epoch [2/10], Step [30/124], Loss: 0.0000
Epoch [2/10], Step [40/124], Loss: 0.0000
Epoch [2/10], Step [50/124], Loss: 0.0000
Epoch [2/10], Step [60/124], Loss: 0.0000
Epoch [2/10], Step [70/124], Loss: 0.0000
Epoch [2/10], Step [80/124], Loss

KeyboardInterrupt: ignored